## End to end model:feature extraction

###  1 import libs

In [1]:
from keras.models import *
from keras.layers import *
from keras.applications import *
from keras.preprocessing.image import *
from keras import optimizers
from keras.applications.vgg19 import preprocess_input
from keras.utils.training_utils import multi_gpu_model
import tensorflow as tf

Using TensorFlow backend.


## 2 build pretrained model¶

In [2]:
with tf.device('/cpu:0'):
    input_tensor = Input((224, 224, 3))
    x = Lambda(vgg19.preprocess_input)(input_tensor)

    base_model = VGG19(input_tensor=x, weights='imagenet', include_top=False)
    base_model.summary()

    print("###body###")

    m_out = base_model.output

    flatten = Flatten(name='flatten')(m_out)

    fc1 = Dense(4096, activation='relu', name='fc1')(flatten)
    drop_fc1 = Dropout(0.5, name='drop_fc1')(fc1)

    fc2 = Dense(4096, activation='relu', name='fc2')(drop_fc1)
    drop_fc2 = Dropout(0.5, name='drop_fc2')(fc2)

    predictions = Dense(18, activation='softmax', name='predictions')(drop_fc2)

    for layer in base_model.layers:  
        layer.trainable = False

    model = Model(inputs=base_model.input, outputs=predictions)
    model.summary()
    print("###whole###")

    sgd = optimizers.SGD(lr=0.001, decay=1e-6, momentum=0.9, nesterov=True)

parallel_model = multi_gpu_model(model, gpus=4)
parallel_model.compile(optimizer=sgd, loss='categorical_crossentropy', metrics=['accuracy'])
parallel_model.summary()
print("###parallel###")

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
lambda_1 (Lambda)            (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
__________

### 3 build param

In [4]:
batch_size = 128
epochs = 5

total_imgs_num = 27140 + 3023
train_imgs_num = 27140
valid_imgs_num = 3023

img_size = (224, 224)

### 4 build data & first train

In [5]:
img_gen_t = ImageDataGenerator()

train_generator = img_gen_t.flow_from_directory("/home/tsimage/high_speed_data/tct_data_samesize_0718_224/train", 
                                                img_size, shuffle=True, batch_size=batch_size)

img_gen_v = ImageDataGenerator()
valid_generator = img_gen_v.flow_from_directory("/home/tsimage/high_speed_data/tct_data_samesize_0718_224/valid",
                                                img_size, shuffle=True, batch_size=batch_size)

parallel_model.fit_generator(generator=train_generator, 
                             steps_per_epoch=train_imgs_num//batch_size, 
                             epochs=epochs, 
                             verbose=1,
                             validation_data=valid_generator, 
                             validation_steps=valid_imgs_num//batch_size)

model.save_weights("vgg19_first_train.h5")

Found 27140 images belonging to 18 classes.
Found 3023 images belonging to 18 classes.
Epoch 1/5
212/212 [==============================] - 178s 841ms/step - loss: 8.6532 - acc: 0.3151 - val_loss: 3.4193 - val_acc: 0.4895
Epoch 2/5
212/212 [==============================] - 180s 847ms/step - loss: 2.6254 - acc: 0.4353 - val_loss: 1.5699 - val_acc: 0.5296
Epoch 3/5
212/212 [==============================] - 179s 842ms/step - loss: 1.7317 - acc: 0.5042 - val_loss: 1.4028 - val_acc: 0.5652
Epoch 4/5
212/212 [==============================] - 180s 847ms/step - loss: 1.4481 - acc: 0.5606 - val_loss: 1.3207 - val_acc: 0.5924
Epoch 5/5
212/212 [==============================] - 180s 848ms/step - loss: 1.2712 - acc: 0.5971 - val_loss: 1.3121 - val_acc: 0.5961


### 5 build finetune model

In [6]:
with tf.device('/cpu:0'):
    input_tensor = Input((224, 224, 3))
    x = Lambda(vgg19.preprocess_input)(input_tensor)

    base_model = VGG19(input_tensor=x, weights='imagenet', include_top=False)
    base_model.summary()

    print("###body###")

    m_out = base_model.output

    flatten = Flatten(name='flatten')(m_out)

    fc1 = Dense(4096, activation='relu', name='fc1')(flatten)
    drop_fc1 = Dropout(0.5, name='drop_fc1')(fc1)

    fc2 = Dense(4096, activation='relu', name='fc2')(drop_fc1)
    drop_fc2 = Dropout(0.5, name='drop_fc2')(fc2)

    predictions = Dense(18, activation='softmax', name='predictions')(drop_fc2)

    model = Model(inputs=base_model.input, outputs=predictions)
    model.load_weights('vgg19_first_train.h5')
    model.summary()
    print("###whole###")

    sgd = optimizers.SGD(lr=0.001, decay=1e-6, momentum=0.9, nesterov=True)

parallel_model = multi_gpu_model(model, gpus=4)
parallel_model.compile(optimizer=sgd, loss='categorical_crossentropy', metrics=['accuracy'])
parallel_model.summary()
print("###parallel###")

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
lambda_6 (Lambda)            (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
__________

### 6 build param

In [7]:
batch_size = 128
epochs = 100

total_imgs_num = 27140 + 3023
train_imgs_num = 27140
valid_imgs_num = 3023

img_size = (224, 224)

### 7 build data & finetune train

In [9]:
img_gen_t = ImageDataGenerator(rotation_range=180,
                               width_shift_range=0.3,
                               height_shift_range=0.3,
                               zoom_range=0.3,
                               horizontal_flip=True,
                               vertical_flip=True)

train_generator = img_gen_t.flow_from_directory("/home/tsimage/high_speed_data/tct_data_samesize_0718_224/train", 
                                                img_size, shuffle=True, batch_size=batch_size)

img_gen_v = ImageDataGenerator()
valid_generator = img_gen_v.flow_from_directory("/home/tsimage/high_speed_data/tct_data_samesize_0718_224/valid", 
                                                img_size, shuffle=True, batch_size=batch_size)


parallel_model.fit_generator(generator=train_generator, 
                             steps_per_epoch=train_imgs_num//batch_size, 
                             epochs=epochs, 
                             verbose=1,
                             validation_data=valid_generator, 
                             validation_steps=valid_imgs_num//batch_size)

model.save_weights("vgg19_finetune.h5")

Found 27140 images belonging to 18 classes.
Found 3023 images belonging to 18 classes.
Epoch 1/100
212/212 [==============================] - 236s 1s/step - loss: 2.2388 - acc: 0.3001 - val_loss: 1.1988 - val_acc: 0.5934
Epoch 2/100
212/212 [==============================] - 234s 1s/step - loss: 1.4135 - acc: 0.5427 - val_loss: 0.9035 - val_acc: 0.6926
Epoch 3/100
212/212 [==============================] - 234s 1s/step - loss: 1.2086 - acc: 0.6104 - val_loss: 0.7923 - val_acc: 0.7340
Epoch 4/100
212/212 [==============================] - 234s 1s/step - loss: 1.0942 - acc: 0.6454 - val_loss: 0.7036 - val_acc: 0.7680
Epoch 5/100
212/212 [==============================] - 234s 1s/step - loss: 0.9987 - acc: 0.6797 - val_loss: 0.6654 - val_acc: 0.7785
Epoch 6/100
212/212 [==============================] - 233s 1s/step - loss: 1.0252 - acc: 0.6711 - val_loss: 1.1594 - val_acc: 0.6433
Epoch 7/100
212/212 [==============================] - 233s 1s/step - loss: 1.0440 - acc: 0.6655 - val_loss: 

212/212 [==============================] - 232s 1s/step - loss: 0.4376 - acc: 0.8546 - val_loss: 0.3701 - val_acc: 0.8818
Epoch 62/100
212/212 [==============================] - 232s 1s/step - loss: 0.4206 - acc: 0.8601 - val_loss: 0.3186 - val_acc: 0.8947
Epoch 63/100
212/212 [==============================] - 232s 1s/step - loss: 0.4053 - acc: 0.8671 - val_loss: 0.3305 - val_acc: 0.8879
Epoch 64/100
212/212 [==============================] - 232s 1s/step - loss: 0.3880 - acc: 0.8689 - val_loss: 0.3214 - val_acc: 0.8903
Epoch 65/100
212/212 [==============================] - 232s 1s/step - loss: 0.3946 - acc: 0.8670 - val_loss: 0.3193 - val_acc: 0.8954
Epoch 66/100
212/212 [==============================] - 231s 1s/step - loss: 0.3833 - acc: 0.8728 - val_loss: 0.3163 - val_acc: 0.8995
Epoch 67/100
212/212 [==============================] - 232s 1s/step - loss: 0.3845 - acc: 0.8706 - val_loss: 0.3288 - val_acc: 0.8967
Epoch 68/100
212/212 [==============================] - 231s 1s/step